<a href="https://colab.research.google.com/github/ArelyA/CE888_2020_Data-Science/blob/master/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [0]:
data = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv", header=None)
data = data.drop(data.columns[0], axis=1)

In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
arr = data.values
rated = np.where(arr!=99)
newdata, x_y = replace(data.values, 0.1)
x, y = x_y

In [0]:
user_ratings = newdata
latent_user_preferences = np.random.random((user_ratings.shape[0], 4))
latent_item_features = np.random.random((user_ratings.shape[1], 4))

In [0]:
def predict_rating(joke_id, rating_id):
    """ Predict a rating given a joke_id and an rating_id.
    """
    user_preference = latent_user_preferences[joke_id]
    item_preference = latent_item_features[rating_id]
    return user_preference.dot(item_preference)


def train(joke_id, rating_id, rating, alpha=0.0001):
    
    #print rating_id
    prediction_rating = predict_rating(joke_id, rating_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[joke_id][:]
    latent_user_preferences[joke_id] -= alpha * err * latent_item_features[rating_id]
    latent_item_features[rating_id] -= alpha * err * user_pref_values
    return err
    

def sgd(al, iterations=50):
    """ Iterate over all jokes and all ratings and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for joke_id in range(latent_user_preferences.shape[0]):
            for rating_id in range(latent_item_features.shape[0]):
                rating = user_ratings[joke_id][rating_id]
                if rating != 99:
                    err = train(joke_id, rating_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()  
        if (iteration % 10) == 0:
            print(mse)

In [0]:
def latent_factor(iter, a, lat):
  # Initialise as random values
  latent_user_preferences = np.random.random((user_ratings.shape[0], lat))
  latent_item_features = np.random.random((user_ratings.shape[1], lat))
  print("MSE %f iter, alpha=%f, latent = %f:" % (iter, a, lat)) 
  sgd(a, iter)

In [8]:
latent_factor(50, 0.0001, 4)

MSE 50.000000 iter, alpha=0.000100, latent = 4.000000:
25.77174859069468
23.945551204368634
21.46278618622512
17.89908151473968
17.39105345884068


In [9]:
latent_factor(50, 0.00001, 4)

MSE 50.000000 iter, alpha=0.000010, latent = 4.000000:
17.222706212834385
17.088317247243534
16.905847030696993
16.632296857808697
16.29566996599351


In [10]:
latent_factor(50, 0.0000001, 2)

MSE 50.000000 iter, alpha=0.000000, latent = 2.000000:
15.952778978251951
15.666372960796574
15.474361639894585
15.360860928024684
15.295828921821208


In [11]:
latent_factor(70, 0.0001, 2)

MSE 70.000000 iter, alpha=0.000100, latent = 2.000000:
15.258057483202041
15.235511807150234
15.221671896082563
15.212969654537085
15.20739806645196
15.203792824113032
15.201458115977559


In [12]:
latent_factor(50, 0.0001, 6)

MSE 50.000000 iter, alpha=0.000100, latent = 6.000000:
15.19996661416273
15.1990493656257
15.198533376003615
15.198305229024363
15.198289321161834
